OpenMLDB sdk init: connect to cluster, and register for sql magic
Plz do `/work/init.sh` to create the OpenMLDB cluster, and `python3 /work/talkingdata/predict_server.py --no-init > predict.log 2>&1 &` to start the simple predict server(receive the deployed sql and model, request it to do online feature extraction)

In [ ]:
import openmldb
db = openmldb.dbapi.connect(zk='127.0.0.1:2181',zkPath='/openmldb')
openmldb.sql_magic.register(db)

The database and table name, which will be used later

In [ ]:
db_name="demo_db"
table_name="talkingdata"

You can use variables like:

In [ ]:
var='1'
%sql SELECT {var};
%sql SELECT $var;

Create database and table(talking data schema)

In [ ]:
%sql create database if not exists $db_name;
%sql use $db_name;
%sql create table if not exists $table_name (ip int, app int, device int, os int, channel int, click_time timestamp, is_attributed int, click_id int, hour int, day int);

Offline load data and extract feature

In [ ]:
%sql set @@execute_mode='offline';
%sql set @@sync_job=true;

In [ ]:
%sql load data infile 'file:///work/talkingdata/train_sample.csv' into table $table_name options(mode='overwrite');

In [ ]:
sql_part = f"""
select is_attributed, app, device, os, channel, hour(click_time) as hour, day(click_time) as day, 
count(channel) over w1 as qty, 
count(channel) over w2 as ip_app_count, 
count(channel) over w3 as ip_app_os_count  
from {table_name} 
window 
w1 as (partition by ip order by click_time ROWS_RANGE BETWEEN 1h PRECEDING AND CURRENT ROW), 
w2 as(partition by ip, app order by click_time ROWS_RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),
w3 as(partition by ip, app, os order by click_time ROWS_RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
"""

train_feature_dir='/tmp/train_feature'

In [ ]:
%sql {sql_part} INTO OUTFILE '{train_feature_dir}' OPTIONS(mode='overwrite');

Train: we use a simple train script to do it, and save the model to 'model_path'

In [ ]:
import xgboost_train_sample
model_path='/tmp/model.json'
xgboost_train_sample.train(f'{train_feature_dir}/*.csv', model_path)

Deploy sql & model, and load data in online mode

In [ ]:
%sql SET @@execute_mode='online';

In [ ]:
deploy_name='d1'
%sql DEPLOY $deploy_name $sql_part;

In [ ]:
%sql load data infile 'file:///work/talkingdata/train_sample.csv' into table $table_name options(mode='append');

Let the predict server know the sql and model

In [ ]:
import requests
predict_server='localhost:8881'
infos = {'database': db_name, 'deployment': deploy_name, 'model_path': model_path}
res = requests.post('http://' + predict_server + '/update', json=infos)
res.text

Then you can request the predict server to test
`python3 predict.py`